# 1. Download docker

https://docs.docker.com/get-started/

2GB is the default memory setting on a fresh installation of Docker for Mac. We [MRIQC developers] recommend increasing the available memory for Docker containers

Docker Preferences > Resources > Memory


The storage location of Docker images and containers
A Docker container consists of network settings, volumes, and images. The location of Docker files depends on your operating system. Here is an overview for the most used operating systems:

Ubuntu: /var/lib/docker/  
Fedora: /var/lib/docker/  
Debian: /var/lib/docker/  
Windows: C:\ProgramData\DockerDesktop  
MacOS: ~/Library/Containers/com.docker.docker/Data/vms/0/  

In macOS and Windows, Docker runs Linux containers in a virtual environment. Therefore, there are some additional things to know.

### Docker for Mac
Docker is not natively compatible with macOS, so Hyperkit is used to run a virtual image. Its virtual image data is located in:  

~/Library/Containers/com.docker.docker/Data/vms/0

Within the virtual image, the path is the default Docker path /var/lib/docker.


### Options usually used when running docker

```
docker run --help
 -i, --interactive                    Keep STDIN open even if not attached
 -t, --tty                            Allocate a pseudo-TTY
 -v, --volume list                    Bind mount a volume
     --rm                             Automatically remove the container when it exits
```

The `-it` runs Docker interactively (so you get a pseudo-TTY with STDIN). The `--rm` causes Docker to automatically remove the container when it exits.

The image being used to create the container is generally specified as `<name>:<tag>` such as ruby:latest. If the specified image is not available locally, Docker will attempt to retrieve it from Docker Hub (or any connected Docker registry).

If you need to find Docker images available locally, you can run: `docker images` or `docker image ls`.

In [ ]:
%%bash

# interact with files in docker

docker ps # get docker container id
docker exec -it d5b73d678398 <command>

docker exec -it \
  -v ~/Downloads/17-1290_TMII_XNAT_S12035_ME-ICA-no-gs:/data d5b73d678398 \
  which 3dFWHMx
  
docker run -ti -d nipreps/fmriprep:20.2.3 \
    -v ~/Downloads/17-1290_TMII_XNAT_S12035_ME-ICA-no-gs:/data \
    3dFWHMx -automask  -input data/17-1290_TMII_XNAT_S12035_TMII_XNAT_E24721_scans_21_T1c_medn_nat.nii.gz



# 2. Download data from XNAT


In [1]:
# conda install -c conda-forge xnat
# TURNED THIS XNAT DOWNLOADER INTO A PYTHON SCRIPT
# /Users/TienTong/Documents/dcm2bids/xnat_download.py

import xnat
import pandas as pd

session = xnat.connect(server='https://tmii02.mssm.edu/xnat', user='tongt03',  verify=False)
project = session.projects["17-1290"]
subjects = project.subjects

# subject = subjects[0]
# subject.download_dir('/Users/TienTong/Documents/dcm2bids/DICOM')

subjects.keys()

keys=[]
values=[]
for key in subjects.keys():
    keys.append(key)
for value in subjects.values():
    values.append(value)
    

[WARNING] Verify is disabled, this will NOT verify the certificate of SSL connections!
[WARNING] Warnings about invalid certificates will be HIDDEN to avoid spam, but this
[WARNING] means that your connection can be potentially unsafe!


Please enter the password for user 'tongt03':········


In [2]:
df = pd.DataFrame(list(zip(keys, values)), columns =['keys', 'values'])
df['subject_id'] = df['values']
for row in range(df.shape[0]):
    df['subject_id'][row] = 'sub-' + str(df['values'][row]).split(" ")[1].split("_")[-1]
    
# df.at[df.subject_id == 'sub-P20013', 'subject_id'] = 'sub-P21002'
   
df.to_csv("/Users/TienTong/Documents/dcm2bids/xnat_allsubjects.csv", header=False)


# 3. Organize the data in BIDS using dcm2bids

BIDS SPECS:
https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/01-magnetic-resonance-imaging-data.html#two-phase-images-and-two-magnitude-images

Why is BIDS so important: https://andysbrainbook.readthedocs.io/en/latest/OpenScience/OS/BIDS_Overview.html

How to convert data to BIDS format: dcm2bids github https://github.com/UNFmontreal/Dcm2Bids


In [ ]:
%%bash
# conda install -c conda-forge dcm2niix
# conda install -c conda-forge dcm2bids

## TEST, SKIP THIS run using dcm2niix only
# dcm2niix \
#    -z y \
#    -f %p_%t_%s \
#    -o /Users/TienTong/Documents/heudiconv/BIDS \
#    /Users/TienTong/Documents/heudiconv/DICOM

# first run `dcm2bids_helper -d /Users/TienTong/Documents/dcm2bids/DICOM/P21013`
# then, modify the dcm2bids_config.json based on the output of the helper
# Example in:
# /Users/TienTong/tmp_dcm2bids/helper

# COMMENT THIS OUT BECAUSE I HAVE TURNED THIS BIDS CONVERSION INTO A BASH SCRIPT
# /Users/TienTong/Documents/dcm2bids/bids_conversion.sh

In [71]:
!tree -L 4 /Users/TienTong/Documents/dcm2bids/BIDS/sub-P18001
# tutorial: https://github.com/bids-standard/pybids/blob/master/examples/pybids_tutorial.ipynb

/Users/TienTong/Documents/dcm2bids/BIDS/sub-P18001
├── anat
│   ├── sub-P18001_T1w.json
│   └── sub-P18001_T1w.nii.gz
├── dwi
│   ├── sub-P18001_dir-LR_dwi.bval
│   ├── sub-P18001_dir-LR_dwi.bvec
│   ├── sub-P18001_dir-LR_dwi.json
│   ├── sub-P18001_dir-LR_dwi.nii.gz
│   ├── sub-P18001_dir-LR_sbref.json
│   ├── sub-P18001_dir-LR_sbref.nii.gz
│   ├── sub-P18001_dir-RL_dwi.bval
│   ├── sub-P18001_dir-RL_dwi.bvec
│   ├── sub-P18001_dir-RL_dwi.json
│   ├── sub-P18001_dir-RL_dwi.nii.gz
│   ├── sub-P18001_dir-RL_sbref.json
│   └── sub-P18001_dir-RL_sbref.nii.gz
├── fmap
│   ├── sub-P18001_dir-AP_epi.json
│   ├── sub-P18001_dir-AP_epi.nii.gz
│   ├── sub-P18001_dir-PA_epi.json
│   └── sub-P18001_dir-PA_epi.nii.gz
└── func
    ├── sub-P18001_task-rest_run-01_bold.json
    ├── sub-P18001_task-rest_run-01_bold.nii.gz
    ├── sub-P18001_task-rest_run-01_echo-01_bold.json
    ├── sub-P18001_task-rest_run-01_echo-01_bold.nii.gz
    ├── sub-P18001_task-rest_run-01_echo-01_sbref.json
    ├── sub-P1800

one full rest scan took ~ 10 minutes

In [29]:
    
#     layout.get_tasks()
# bf_filename = layout.get(subject=layout.get_subjects()[0], 
#            extension='.json', datatype='fmap', return_type='filename')[0]
test = layout.get(subject=layout.get_subjects()[0], 
               extension='.json', datatype='func')[1]

# Print it
print(test)

# Print all the entities associated with this file, and their values
print(test.get_entities())
layout.get(subject=layout.get_subjects()[0], 
               extension='.json', datatype='func', absolute_paths=False, return_type='filename')

<BIDSJSONFile filename='/Users/TienTong/Documents/dcm2bids/BIDS/sub-P18001/func/sub-P18001_task-rest_run-01_echo-01_sbref.json'>
{'datatype': 'func', 'echo': '01', 'extension': 'json', 'run': 1, 'subject': 'P18001', 'suffix': 'sbref', 'task': 'rest'}


/Users/TienTong/opt/anaconda3/lib/python3.8/site-packages/bids/layout/validation.py:37: UserWarning: The absolute_paths argument will be removed from PyBIDS in 0.14. You can easily access the relative path of BIDSFile objects via the .relpath attribute (instead of .path). Switching to this pattern is strongly encouraged, as the current implementation of relative path handling is known to produce query failures in certain edge cases.
  warnings.warn("The absolute_paths argument will be removed from PyBIDS "


['sub-P18001/func/sub-P18001_task-rest_run-01_echo-01_bold.json',
 'sub-P18001/func/sub-P18001_task-rest_run-01_echo-01_sbref.json',
 'sub-P18001/func/sub-P18001_task-rest_run-01_echo-02_bold.json',
 'sub-P18001/func/sub-P18001_task-rest_run-01_echo-02_sbref.json',
 'sub-P18001/func/sub-P18001_task-rest_run-01_echo-03_bold.json',
 'sub-P18001/func/sub-P18001_task-rest_run-01_echo-03_sbref.json',
 'sub-P18001/func/sub-P18001_task-rest_run-02_echo-01_bold.json',
 'sub-P18001/func/sub-P18001_task-rest_run-02_echo-01_sbref.json',
 'sub-P18001/func/sub-P18001_task-rest_run-02_echo-02_bold.json',
 'sub-P18001/func/sub-P18001_task-rest_run-02_echo-02_sbref.json',
 'sub-P18001/func/sub-P18001_task-rest_run-02_echo-03_bold.json',
 'sub-P18001/func/sub-P18001_task-rest_run-02_echo-03_sbref.json',
 'sub-P18001/func/sub-P18001_task-socialnav_bold.json',
 'sub-P18001/func/sub-P18001_task-socialnav_sbref.json',
 'sub-P18001/func/sub-P18001_task-whyhow_bold.json',
 'sub-P18001/func/sub-P18001_task-wh

# 4. Install MRIQC

https://mriqc.readthedocs.io/en/stable/docker.html#docker

```
docker run -it poldracklab/mriqc:latest --version # don't use MRIQC v0.16.1, some commands were deprecated
docker run -it poldracklab/mriqc:0.15.1 --version
docker run -it poldracklab/mriqc:0.14.2 --version

# options usually used when running docker
# docker run --help
#  -i, --interactive                    Keep STDIN open even if not attached
#  -t, --tty                            Allocate a pseudo-TTY
#  -v, --volume list                    Bind mount a volume
#      --rm                             Automatically remove the container when it exits

```

In [ ]:
%%bash

# docker exec -it c31e8ab5c3af pip install pydeface # DID NOT WORK
docker pull poldracklab/pydeface

files=`ls /Users/TienTong/Documents/dcm2bids/BIDS/deface/sub-P19027/*/*.nii.gz`

for f in files ; do
  pydeface $f 
done

In [ ]:
%%bash

# run MRIQC

# docker run -it --rm -v /Users/TienTong/Documents/dcm2bids/BIDS:/data:ro -v /Users/TienTong/Documents/mriqc:/out \
#             --read-only --tmpfs /run --tmpfs /tmp \
#             poldracklab/mriqc:0.14.2 \
#                 /data \
#                 /out \
#                 participant \
#                 --participant_label P21013 \
#                 --no-sub \
#                 -w /out --nprocs 4 \
#                 --verbose-reports > /Users/TienTong/Documents/mriqc/mriqc_log.txt

docker run -it --rm -v /Users/TienTong/Documents/dcm2bids/BIDS:/data:ro -v /Users/TienTong/Documents/mriqc_15:/out \
            --read-only --tmpfs /run --tmpfs /tmp \
            poldracklab/mriqc:0.15.1 \
                /data \
                /out \
                participant \
                --participant_label P19027 \
                --no-sub \
                -w /out --nprocs 4 \
                --verbose-reports > /Users/TienTong/Documents/mriqc_15/mriqc15_P19027_log.txt


# fMRIPrep

In [ ]:
%%bash

# Installation instructions here: https://www.nipreps.org/apps/docker/

# docker pull nipreps/fmriprep:20.2.3
# pip install --user --upgrade fmriprep-docker

# run
docker run -ti --rm \
    -v $HOME/Documents/dcm2bids/BIDS/sub-P18004:/data:ro \
    -v $HOME/Documents/fmriprep:/out \
    -v $HOME/Documents/fmriprep:/work \
    -v $HOME/Documents/Licenses/freesurfer_license.txt:/opt/freesurfer/license.txt \
    nipreps/fmriprep:20.2.3 \
        /data \
        /out \
        participant \
        -w /work \
        -t rest \
        --output-spaces MNI152NLin6Asym:res-2 \
        --nprocs 4 \
        --skip_bids_validation \
        --use-aroma > ~/Documents/fmriprep/sub-P18004.log
        
        # run time: 9h 26m 10s

In [ ]:
%%bash

docker run -ti --rm \
    -v $HOME/Documents/rest/sub-P18005:/data:ro \
    -v $HOME/Documents/fmriprep:/out \
    -v $HOME/Documents/fmriprep:/work \
    -v $HOME/Documents/Licenses/freesurfer_license.txt:/opt/freesurfer/license.txt \
    nipreps/fmriprep:20.2.3 \
        /data \
        /out \
        participant \
        -w /work \
        -t rest \
        --output-spaces func \
        --nprocs 4 \
        --skip_bids_validation \
        --ignore slicetiming \
        --fs-no-reconall \
        --no-submm-recon \
        --use-aroma > ~/Documents/fmriprep/sub-P18005.log
